<a href="https://colab.research.google.com/github/Rafsan7238/BracU_Thesis_P2/blob/main/Pre_Thesis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Application of Deep Convolutional Neural Network in Breast Cancer Prediction Using Digital Mammograms**

**Authors:** Rafsan Al Mamun, Md. Al Imran Sefat, Gazi Abu Rafin, Adnan

**Project GitHub Link:** [https://github.com/Rafsan7238/BracU_Thesis_P2](https://github.com/Rafsan7238/BracU_Thesis_P2)



---



***Abstract:*** *Cancer, a diagnosis that is so dreaded and scary, that its fear alone can strike even the
strongest of souls. The disease is often thought of as untreatable and unbearably painful, with
usually, no cure available. Among all the cancers, breast cancer is the second most deadliest ,
especially among women. What decides the patients’ fate is the early diagnosis of the cancer,
facilitating subsequent clinical management. Mammography plays a vital role in the
screening of breast cancers as it can detect any breast masses or calcifications early. However,
the extremely dense breast tissues pose difficulty in the detection of cancer mass, thus,
encouraging the use of machine learning (ML) techniques and artificial neural networks
(ANN) to assist radiologists in faster cancer diagnosis. This paper explores the MIAS
database, containing 332 digital mammograms from women, which were augmented and
preprocessed, and fed into different convolutional neural network (CNN) models, with the
aim of differentiating healthy tissues from cancerous ones with high accuracy. The paper,
along with a new proposed CNN model for better identification of breast cancer, focuses on
the significance of computer-aided detection (CAD) models overall in the early diagnosis of
breast cancer. While a diagnosis of breast cancer may still leave patients dreaded, we believe
our research can be a symbol of hope for all.*

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Data Collection and Preprocessing**

In [5]:
pip install -U albumentations

     |████████████████████████████████| 102kB 6.9MB/s 
     |████████████████████████████████| 38.2MB 81kB/s 
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [6]:
import cv2
from google.colab.patches import cv2_imshow
import os
import albumentations as A

def load_data():
  """
    Load image data from directory '/content/drive/MyDrive/Thesis/Dataset'.

    Load each image file from the subdirectories of Dataset, turn it into B/W and augment it. 
    After each augmentation append the images into the images list, and their corresponding
    labels in the labels list. 

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions 224 x 224 x 3. `labels` should
    be a list of labels (0 for healthy or 1 for cancer), representing the categories for each of the
    corresponding `images`.
    """

  images = []
  labels = []

  # Loop through the healthy dataset
  directory_path = "/content/drive/MyDrive/Thesis/Dataset/Healthy"
  os.chdir(directory_path)
  count = 1

  for file in os.listdir():
    if file.endswith(".jpg"):

      print(f"Working with {count}  healthy images out of 272")
      count += 1

      file_path = os.path.join(directory_path, file)
      img = cv2.imread(file_path)
      
      #TODO

      # 1. Resize to 224*224      
      img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
      
      images.append(img)
      labels.append(0)

      # 2. Turn to LAB for CLAHE
      image_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
      lab_planes = cv2.split(image_lab)
      
      # 3. CLAHE
      # The declaration of CLAHE 
      # Apply CLAHE on the luminescence channel
      # clipLimit -> Threshold for contrast limiting
      clahe = cv2.createCLAHE(clipLimit = 5)
      lab_planes[0] = clahe.apply(lab_planes[0])

      # Merge the LAB planes into an LAB image, and convert it back to RGB
      image_lab = cv2.merge(lab_planes)
      final_img = cv2.cvtColor(image_lab, cv2.COLOR_LAB2BGR)

      images.append(final_img)
      labels.append(0)

      # 4. augmentation and append


      # 4.a) rotate 10  
      transform = A.Compose([
        A.Rotate(10)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]

      images.append(transformed_image)
      labels.append(0)

      # 4.b) rotate 20 
      transform = A.Compose([
        A.Rotate(20)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]

      images.append(transformed_image)
      labels.append(0)

      # 4.c) horizontal flip 
      transform = A.Compose([
        A.HorizontalFlip(p=1)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]

      images.append(transformed_image)
      labels.append(0)

       # 4.d) vertical flip 
      transform = A.Compose([
        A.VerticalFlip(p=1)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]

      images.append(transformed_image)
      labels.append(0)

      # 4.e) random tone curve 
      transform = A.Compose([
        A.RandomToneCurve(always_apply = True)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
      
      images.append(transformed_image)
      labels.append(0)

      # 4.f) GaussNoise 
      transform = A.Compose([
        A.GaussNoise()
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
      
      images.append(transformed_image)
      labels.append(0)


      # 4.g) Blur 
      transform = A.Compose([
        A.Blur()
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
     
      images.append(transformed_image)
      labels.append(0)

    

  ##### Same for cancer dataset

  # Loop through the cancer dataset
  directory_path = "/content/drive/MyDrive/Thesis/Dataset/Cancer"
  os.chdir(directory_path)
  count = 1

  for file in os.listdir():
    if file.endswith(".jpg"):
     
      print(f"Working with {count} cancer images out of 50")
      count += 1

      file_path = os.path.join(directory_path, file)
      img = cv2.imread(file_path)
      
      #TODO

      # 1. Resize to 224*224      
      img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
     
      images.append(img)
      labels.append(1)

      # 2. Turn to LAB for CLAHE
      image_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
      lab_planes = cv2.split(image_lab)
      
      # 3. CLAHE
      # The declaration of CLAHE 
      # Apply CLAHE on the luminescence channel
      # clipLimit -> Threshold for contrast limiting
      clahe = cv2.createCLAHE(clipLimit = 5)
      lab_planes[0] = clahe.apply(lab_planes[0])

      # Merge the LAB planes into an LAB image, and convert it back to RGB
      image_lab = cv2.merge(lab_planes)
      final_img = cv2.cvtColor(image_lab, cv2.COLOR_LAB2BGR)

      images.append(final_img)
      labels.append(1)

      # 4. augmentation and append


      # 4.a) rotate 10  
      transform = A.Compose([
        A.Rotate(10)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
      
      images.append(transformed_image)
      labels.append(1)

      # 4.b) rotate 20 
      transform = A.Compose([
        A.Rotate(20)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
      
      images.append(transformed_image)
      labels.append(1)

      # 4.c) horizontal flip 
      transform = A.Compose([
        A.HorizontalFlip(p=1)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
      
      images.append(transformed_image)
      labels.append(1)

       # 4.d) vertical flip 
      transform = A.Compose([
        A.VerticalFlip(p=1)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
     
      images.append(transformed_image)
      labels.append(1)

      # 4.e) random tone curve 
      transform = A.Compose([
        A.RandomToneCurve(always_apply = True)
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
      
      images.append(transformed_image)
      labels.append(1)

      # 4.f) GaussNoise 
      transform = A.Compose([
        A.GaussNoise()
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
     
      images.append(transformed_image)
      labels.append(1)


      # 4.g) Blur 
      transform = A.Compose([
        A.Blur()
      ])
      transformed = transform(image = final_img)
      transformed_image = transformed["image"]
      
      images.append(transformed_image)
      labels.append(1)

  # Return a tuple of (images, labels)

  return (images, labels)

## **Image Visualisation**

In [7]:
images, labels = load_data()

Working with 1  healthy images out of 272
Working with 2  healthy images out of 272
Working with 3  healthy images out of 272
Working with 4  healthy images out of 272
Working with 5  healthy images out of 272
Working with 6  healthy images out of 272
Working with 7  healthy images out of 272
Working with 8  healthy images out of 272
Working with 9  healthy images out of 272
Working with 10  healthy images out of 272
Working with 11  healthy images out of 272
Working with 12  healthy images out of 272
Working with 13  healthy images out of 272
Working with 14  healthy images out of 272
Working with 15  healthy images out of 272
Working with 16  healthy images out of 272
Working with 17  healthy images out of 272
Working with 18  healthy images out of 272
Working with 19  healthy images out of 272
Working with 20  healthy images out of 272
Working with 21  healthy images out of 272
Working with 22  healthy images out of 272
Working with 23  healthy images out of 272
Working with 24  hea

In [8]:
print(len(images))
print(len(labels))

# for i in range(10):
#   cv2_imshow(images[i])

2898
2898


## **Necessary Imports for Models**

In [74]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, EfficientNetB7, MobileNetV3Small, VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.metrics import Accuracy, Precision
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

## **Standalone Transfer Learning Models**

### **Pre-trained ResNet50 Model**

In [45]:
def get_resnet_pretrained():
  base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

  for layer in base_model.layers:
    layer.trainable = False


  x = Flatten()(base_model.output)
  x = Dense(512, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(1, activation='sigmoid')(x)

  model = Model(base_model.input, x)

  model.compile(
      optimizer = "adam",
      loss="binary_crossentropy",
      metrics=["accuracy"]
  )

  return model

### **Pre-trained EfficientNet B7**

In [51]:
def get_efficientnet_pretrained():
  base_model = EfficientNetB7(input_shape=(224, 224,3), include_top=False, weights="imagenet")

  for layer in base_model.layers:
    layer.trainable = False

  x = Flatten()(base_model.output)
  x = Dense(512, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(1, activation='sigmoid')(x)

  model = Model(base_model.input, x)

  model.compile(
      optimizer = "adam",
      loss="binary_crossentropy",
      metrics=["accuracy"]
  )

  return model

### **Pre-trained MobileNet V3 Small Model**

In [72]:
def get_mobilenet_pretrained():
  base_model = MobileNetV3Small(input_shape=(224, 224,3), include_top=False, weights="imagenet")

  for layer in base_model.layers:
    layer.trainable = False

  x = Flatten()(base_model.output)
  x = Dense(512, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(1, activation='sigmoid')(x)

  model = Model(base_model.input, x)

  model.compile(
      optimizer = "adam",
      loss="binary_crossentropy",
      metrics=["accuracy"]
  )

  return model

### **Pre-trained VGG16 Model**

In [79]:
def get_vgg_pretrained():
  base_model = VGG16(input_shape=(224, 224,3), include_top=False, weights="imagenet")

  for layer in base_model.layers:
    layer.trainable = False

  x = Flatten()(base_model.output)
  x = Dense(512, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(1, activation='sigmoid')(x)

  model = Model(base_model.input, x)

  model.compile(
      optimizer = "adam",
      loss="binary_crossentropy",
      metrics=["accuracy"]
  )

  return model

## **Main Method**

**Split the Data into Train and Test Sets**

In [12]:
x_train, x_test, y_train, y_test = train_test_split(np.array(images), np.array(labels), test_size=0.2, shuffle=True)

### **Pre-trained ResNet50**

**Train and Test**

In [52]:
pretrained_resnet_model = get_resnet_pretrained()
pretrained_resnet_model.fit(x_train, y_train, validation_split=0.1, epochs=10, steps_per_epoch=100)
pretrain_resnet50_history = pretrained_resnet_model.evaluate(x_test, y_test, verbose=2)
pretrained_resnet_y_pred = pretrained_resnet_model.predict(x_test).ravel()

Epoch 1/10
100/100 [==============================] - 12s 91ms/step - loss: 9.9126 - accuracy: 0.7939 - val_loss: 1.3556 - val_accuracy: 0.8190
Epoch 2/10
100/100 [==============================] - 8s 84ms/step - loss: 0.3546 - accuracy: 0.9046 - val_loss: 0.2739 - val_accuracy: 0.9181
Epoch 3/10
100/100 [==============================] - 9s 85ms/step - loss: 0.2256 - accuracy: 0.9267 - val_loss: 0.2543 - val_accuracy: 0.9138
Epoch 4/10
100/100 [==============================] - 9s 86ms/step - loss: 0.1729 - accuracy: 0.9406 - val_loss: 0.3362 - val_accuracy: 0.9052
Epoch 5/10
100/100 [==============================] - 9s 86ms/step - loss: 0.1445 - accuracy: 0.9506 - val_loss: 0.2589 - val_accuracy: 0.9224
Epoch 6/10
100/100 [==============================] - 8s 85ms/step - loss: 0.1651 - accuracy: 0.9406 - val_loss: 0.1846 - val_accuracy: 0.9440
Epoch 7/10
100/100 [==============================] - 8s 84ms/step - loss: 0.1162 - accuracy: 0.9525 - val_loss: 0.2911 - val_accuracy: 0.922

**Evaluation**

### **Pre-trained EfficientNet B7**

**Train and Test**

In [53]:
pretrained_efficientnet_model = get_efficientnet_pretrained()
pretrained_efficientnet_model.fit(x_train, y_train, validation_split=0.1, epochs=10, steps_per_epoch=100)
pretrain_efficientnet_history = pretrained_efficientnet_model.evaluate(x_test, y_test, verbose=2)
pretrained_efficientnet_y_pred = pretrained_efficientnet_model.predict(x_test).ravel()

Epoch 1/10
100/100 [==============================] - 60s 318ms/step - loss: 4.7346 - accuracy: 0.7886 - val_loss: 0.3348 - val_accuracy: 0.8578
Epoch 2/10
100/100 [==============================] - 28s 283ms/step - loss: 0.2986 - accuracy: 0.8907 - val_loss: 0.2674 - val_accuracy: 0.9095
Epoch 3/10
100/100 [==============================] - 29s 287ms/step - loss: 0.2387 - accuracy: 0.9137 - val_loss: 0.2511 - val_accuracy: 0.9095
Epoch 4/10
100/100 [==============================] - 29s 286ms/step - loss: 0.1948 - accuracy: 0.9243 - val_loss: 0.2425 - val_accuracy: 0.9095
Epoch 5/10
100/100 [==============================] - 28s 285ms/step - loss: 0.1611 - accuracy: 0.9367 - val_loss: 0.2558 - val_accuracy: 0.9181
Epoch 6/10
100/100 [==============================] - 29s 286ms/step - loss: 0.1307 - accuracy: 0.9511 - val_loss: 0.2650 - val_accuracy: 0.8922
Epoch 7/10
100/100 [==============================] - 29s 286ms/step - loss: 0.1220 - accuracy: 0.9588 - val_loss: 0.2077 - val_ac

**Evaluation**

### **Pre-trained MobileNet V3 Small**

**Train and Test**

In [73]:
pretrained_mobilenet_model = get_mobilenet_pretrained()
pretrained_mobilenet_model.fit(x_train, y_train, validation_split=0.1, epochs=8)
pretrain_mobilenet_history = pretrained_mobilenet_model.evaluate(x_test, y_test, verbose=2)
pretrained_mobilenet_y_pred = pretrained_mobilenet_model.predict(x_test).ravel()

Epoch 1/8
66/66 [==============================] - 5s 37ms/step - loss: 5.9872 - accuracy: 0.7852 - val_loss: 0.4595 - val_accuracy: 0.8664
Epoch 2/8
66/66 [==============================] - 2s 24ms/step - loss: 0.2560 - accuracy: 0.9036 - val_loss: 0.3486 - val_accuracy: 0.8750
Epoch 3/8
66/66 [==============================] - 2s 24ms/step - loss: 0.1945 - accuracy: 0.9310 - val_loss: 0.2977 - val_accuracy: 0.8879
Epoch 4/8
66/66 [==============================] - 2s 24ms/step - loss: 0.1343 - accuracy: 0.9487 - val_loss: 0.2098 - val_accuracy: 0.9310
Epoch 5/8
66/66 [==============================] - 2s 24ms/step - loss: 0.1526 - accuracy: 0.9497 - val_loss: 0.1353 - val_accuracy: 0.9483
Epoch 6/8
66/66 [==============================] - 2s 24ms/step - loss: 0.0869 - accuracy: 0.9708 - val_loss: 0.2296 - val_accuracy: 0.9353
Epoch 7/8
66/66 [==============================] - 2s 24ms/step - loss: 0.0610 - accuracy: 0.9784 - val_loss: 0.3004 - val_accuracy: 0.9267
Epoch 8/8
66/66 [===

**Evaluation**

### **Pre-trained VGG16**

**Train and Test**

In [84]:
pretrained_vgg_model = get_vgg_pretrained()
pretrained_vgg_model.fit(x_train, y_train, validation_split=0.1, epochs=6)
pretrain_vgg_history = pretrained_vgg_model.evaluate(x_test, y_test, verbose=2)
pretrained_vgg_y_pred = pretrained_vgg_model.predict(x_test).ravel()

Epoch 1/6
66/66 [==============================] - 10s 141ms/step - loss: 7.1143 - accuracy: 0.7891 - val_loss: 0.2989 - val_accuracy: 0.9224
Epoch 2/6
66/66 [==============================] - 9s 140ms/step - loss: 0.2639 - accuracy: 0.9032 - val_loss: 0.2189 - val_accuracy: 0.9267
Epoch 3/6
66/66 [==============================] - 9s 141ms/step - loss: 0.1719 - accuracy: 0.9338 - val_loss: 0.2032 - val_accuracy: 0.9483
Epoch 4/6
66/66 [==============================] - 9s 142ms/step - loss: 0.1370 - accuracy: 0.9468 - val_loss: 0.2461 - val_accuracy: 0.9440
Epoch 5/6
66/66 [==============================] - 9s 143ms/step - loss: 0.1126 - accuracy: 0.9607 - val_loss: 0.2452 - val_accuracy: 0.9483
Epoch 6/6
66/66 [==============================] - 9s 143ms/step - loss: 0.0921 - accuracy: 0.9674 - val_loss: 0.2379 - val_accuracy: 0.9353
19/19 - 2s - loss: 0.2512 - accuracy: 0.9431


**Evaluation**